Using prefix sums to compute range sums is efficient when the array is static because a single prefix sum array allows constant-time queries for any range. However, when the array can be updated frequently, prefix sums become inefficient because every update would require recomputing the prefix sums for all subsequent elements, leading to O(n) time per update. This is why data structures like the Fenwick Tree and segment trees were created: they allow both point updates and range sum queries in O(log n) time, efficiently handling dynamic arrays without recomputing the entire prefix sum after each change.

Fenwick Tree itself is just another array, that stores ranges in each cell. 

In a Fenwick Tree, each node (cell of the array) at index i is responsible for a contiguous range of elements in the original array, and the size of this range is determined by the least significant bit of i. To find the range that a node covers, you compute the LSB using (we can use i & -i, this is a shortcut to get LSB). The range of the original array that tree[i] stores is from i - lsb(i) + 1 to i inclusive. Fenwick Trees are implemented as 1-indexed arrays, so the first index tree[0] is left unused because the LSB of 0 is 0, which would cause infinite loops in the update and prefix sum operations. Every element of the original array at index j corresponds to the node at index j + 1 in the tree, ensuring that the LSB calculations and range sums work correctly.

In [4]:
def ft_range(i):
    lsb = i & -i
    return (i - lsb + 1, i)
    
for i in range(1, 8):
    print(f"tree[{i}] covers indices {ft_range(i)}")

tree[1] covers indices (1, 1)
tree[2] covers indices (1, 2)
tree[3] covers indices (3, 3)
tree[4] covers indices (1, 4)
tree[5] covers indices (5, 5)
tree[6] covers indices (5, 6)
tree[7] covers indices (7, 7)


**Transversing a Fenwick Tree**

tree[1] covers indices (1, 1)
tree[2] covers indices (1, 2)
tree[3] covers indices (3, 3)
tree[4] covers indices (1, 4)
tree[5] covers indices (5, 5)
tree[6] covers indices (5, 6)
tree[7] covers indices (7, 7)

In [ ]:
class FenwickTree:
    def __init__(self, array):
        self.array = array
        self.size = len(array)
        self.tree = [0] + self.array[:]  # 1-based indexing

        # build FT in O(n)
        for i in range(1, self.size + 1):
            # move to next node of the fenwick tree and add the element it contains.
            j = i + self.lsb(i)
            if j <= self.size:
                self.tree[j] += self.tree[i]

    # lsb to get increment for next or previous index
    def lsb(self, n):
        return n & -n

    # Prefix sum from index 0 to idx
    def prefix_sum(self, idx):
        idx += 1  # convert to 1-based
        result = 0
        # start at that index and go backward cummulating the ranges to get prefix sum
        while idx > 0:
            result += self.tree[idx]
            idx -= self.lsb(idx)
        return result

    # Update element at idx to new_val
    def update(self, idx, new_val):
        idx += 1  # convert to 1-based
        # update diff is what we need to add to that value to make it equal to our update.
        update_diff = new_val - self.array[idx - 1]
        self.array[idx - 1] = new_val
        # go forward to other ranges containing the update and propagate the update diff
        while idx <= self.size:
            self.tree[idx] += update_diff
            idx += self.lsb(idx)

    # Range sum from a to b
    def range_sum(self, a, b):
        if a > b:
            return -1
        return self.prefix_sum(b) - self.prefix_sum(a - 1)